#### Важное требование ко всей домашке в целом: в jupyter ноутбуке не должно был лишнего кода (т.е. если вы взяли за основу семинар, не забудьте удалить все лишнее)

In [50]:
import tensorflow as tf
import gensim
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances
from pymystem3 import Mystem
import json
import random
import pickle

! pip install pymorphy2
from pymorphy2 import MorphAnalyzer

Корпус с Wikipedia в качестве обучающих данных

In [5]:
wiki_texts = open('wiki_data.txt').read().split('\n')

Чтобы по несколько раз не делать долгую предобработку с лемматизацией, предобработанные тексты были сохранены в pickle файл

In [6]:
with open('wiki_texts_preprocessed.pickle', 'rb') as f:
    wiki_texts_preprocessed = pickle.load(f)

In [100]:
wiki_texts_preprocessed[0][45:50] # слова приведены к нормальной форме

['соединить', 'асфальтовый', 'дорога', 'с', 'посёлок']

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window не так используется)  
3) измените часть с np.random.randint(vocab_size) так, чтобы случайные негативные примеры выбирались обратно пропорционально частотностям слов (частотные должны выбираться реже, а редкие чаще)

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [36]:
# поиск векторов с минимальным косинусным расстоянием
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [16]:
morph = MorphAnalyzer()

# предобработка -> токенизация + лемматизация
def preprocess(text):
  tokens = text.lower().split()
  tokens = [token.strip(punctuation) for token in tokens]
  # лемматизируем с pymorphy
  tokens = [morph.parse(token)[0].normal_form for token in tokens]
  return(tokens)

In [18]:
vocab = Counter()

for text in wiki_texts_preprocessed:
    vocab.update(text)

In [19]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

In [20]:
len(filtered_vocab)

11987

Создаем словарь с индексами слов

In [21]:
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
    
id2word = {i:word for word, i in word2id.items()}

In [22]:
sentences = []

for text in wiki_texts_preprocessed:
    tokens = text
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

Создадим словарь из пар *id_слова + частота*, чтобы учитывать частоту слов при выборе отрицательных примеров для negative sampling:

In [25]:
filtered_vocab_counts = {}

for key in vocab.keys():
    if key in filtered_vocab:
      filtered_vocab_counts[key] = vocab[key]

In [26]:
list(filtered_vocab_counts.items())[150:155]

[('выделить', 628),
 ('относительно', 443),
 ('крупный', 2381),
 ('весь', 7838),
 ('обозначить', 172)]

In [27]:
filtered_vocab_probs_ids= {}

n = len(filtered_vocab_counts)
for key in filtered_vocab_counts:
  if key in word2id.keys():
      filtered_vocab_probs_ids[word2id[key]] = filtered_vocab_counts[key]/n

In [28]:
list(filtered_vocab_probs_ids.items())[150:155]

[(10779, 0.05239008926336865),
 (2074, 0.036956703095019604),
 (197, 0.1986318511721031),
 (7031, 0.6538750312838909),
 (11804, 0.014348877951113706)]

### Skip Gram Negative Sampling

In [29]:
# skip gram, генерируем обучающие примеры батчами
def gen_batches_sg(sentences, window, batch_size=1000):
    
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]

                # задаем контекст, делим размер окна на 2 
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2]

                for context_word in context:
                    # генерируем положительные примеры
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)
                    
                    # генерируем отрицательные примеры
                    X_target.append(word)    
                    # генерируем 50 рандомных индексов с учетом частот
                    id_based_on_frequency = random.choices(list(filtered_vocab_probs_ids.keys()), 
                                                           weights=filtered_vocab_probs_ids.values(), 
                                                           k=50)[np.random.randint(50)] # берем один рандомный индекс из 50-ти
                    X_context.append(id_based_on_frequency) 
                    y.append(0)
                    
                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [30]:
inputs_target = tf.keras.layers.Input(shape=(1,)) # input для целевого слова
inputs_context = tf.keras.layers.Input(shape=(1,)) # input для контекстного слова


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)

dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

skipgram_neg_smp_model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
skipgram_neg_smp_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
skipgram_neg_smp_model.fit(gen_batches_sg(sentences[:19000], window=12),
          validation_data=gen_batches_sg(sentences[19000:],  window=12),
          batch_size=1000,
          steps_per_epoch=3000,
          validation_steps=30,
          epochs=2)

Epoch 1/2
3000/3000 [==============================] - 1070s 357ms/step - loss: 0.6764 - accuracy: 0.5577 - val_loss: 0.6749 - val_accuracy: 0.5709
Epoch 2/2
3000/3000 [==============================] - 1123s 374ms/step - loss: 0.6637 - accuracy: 0.5829 - val_loss: 0.6831 - val_accuracy: 0.5540


In [51]:
embeddings_skipgram_negsmplg =  skipgram_neg_smp_model.layers[2].get_weights()[0]

In [52]:
most_similar('город', embeddings_skipgram_negsmplg)

['город',
 'скотсдейл',
 'провинция',
 'северо-запад',
 'онтарио',
 'расположить',
 'деревня',
 'напрямую',
 'полуостров',
 'район']

In [58]:
most_similar('школа', embeddings_skipgram_negsmplg)

['школа',
 'учиться',
 'окончить',
 'бакалавр',
 'преподавать',
 'университет',
 'училище',
 'московский',
 'факультет',
 'поступить']

### CBOW negative sampling

In [32]:
# cbow 
def gen_batches_cbow(sentences, window, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                # делим размер окна на 2
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2]

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                id_based_on_frequency = random.choices(list(filtered_vocab_probs_ids.keys()), 
                                                           weights=filtered_vocab_probs_ids.values(), 
                                                           k=50)[np.random.randint(50)]
                X_target.append(np.random.randint(id_based_on_frequency))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window*2)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [33]:
#cbow negative sampling
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model.fit(gen_batches_cbow(sentences[:19000], window=6),
          validation_data=gen_batches_cbow(sentences[19000:],  window=6),
          batch_size=1000,
          steps_per_epoch=3000,
          validation_steps=30,
          epochs=2)

Epoch 1/2
3000/3000 [==============================] - 1028s 343ms/step - loss: 0.3343 - accuracy: 0.8572 - val_loss: 0.2957 - val_accuracy: 0.8777
Epoch 2/2
3000/3000 [==============================] - 1033s 344ms/step - loss: 0.2732 - accuracy: 0.8885 - val_loss: 0.2552 - val_accuracy: 0.8954


In [37]:
embeddings_cbow_negsmplg = model.layers[2].get_weights()[0]

In [54]:
most_similar('город', embeddings_cbow_negsmplg)

['город',
 'район',
 'посёлок',
 'улица',
 'ростов',
 'здание',
 'монастырь',
 'столица',
 'деревня',
 'финляндия']

In [57]:
most_similar('школа', embeddings_cbow_negsmplg)

['школа',
 'училище',
 'университет',
 'институт',
 'семинария',
 'мастерская',
 'учиться',
 'учитель',
 'музей',
 'колледж']

**Вывод**: если судить по полученным похожим словам, обе модели генерируют достаточно хорошие эмбеддинги. Функция потерь для модели CBOW минимизировалась быстрее, чем для Skip-Gram (при равном числе эпох и размере батча). Это может быть связано с размером окна, который для CBOW был в два раза меньше, чем у Skip-Gram (но лучше проверить)

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [ ]:
# texts = [preprocess(text) for text in wiki_texts]

In [59]:
texts = wiki_texts_preprocessed

### Word2Vec

In [60]:
%%time
w2v = gensim.models.Word2Vec(texts, 
                             size=128, 
                             min_count=25, 
                             max_vocab_size=11000,
                             window=7,
                             iter=8,
                             sg=1,
                             hs=0,
                             negative=10,
                             sample=1e-4,
                             ns_exponent=0.5,
                             cbow_mean=1)

CPU times: user 5min 7s, sys: 1.18 s, total: 5min 8s
Wall time: 2min 56s


In [61]:
w2v.wv.most_similar('город')

[('столица', 0.7060852646827698),
 ('центр', 0.6457962989807129),
 ('посёлок', 0.623887300491333),
 ('район', 0.5897524356842041),
 ('городок', 0.5703724026679993),
 ('городской', 0.5620348453521729),
 ('коммуна', 0.5583091974258423),
 ('расположить', 0.5504782199859619),
 ('пункт', 0.5489876866340637),
 ('административный', 0.5486613512039185)]

In [62]:
w2v.wv.most_similar('школа')

[('учиться', 0.7955573201179504),
 ('обучаться', 0.7156641483306885),
 ('училище', 0.7023470401763916),
 ('учитель', 0.6892426013946533),
 ('окончить', 0.6658641695976257),
 ('колледж', 0.6552792191505432),
 ('обучение', 0.6394620537757874),
 ('преподаватель', 0.629569411277771),
 ('преподавать', 0.627708911895752),
 ('факультет', 0.6234071254730225)]

### FastText

In [63]:
ft = gensim.models.FastText(texts, min_n=4, max_n=9)

Для FastText модели можно сразу заметить влияние n-грамм:

In [64]:
ft.wv.most_similar('город')

[('ужгород', 0.9822290539741516),
 ('городе»', 0.9706169366836548),
 ('«город', 0.9700791835784912),
 ('город»', 0.9697728157043457),
 ('горо', 0.9651158452033997),
 ('городец', 0.9648722410202026),
 ('горох', 0.9550338387489319),
 ('городов', 0.9530773758888245),
 ('огород', 0.9500880241394043),
 ('белгород', 0.9493427872657776)]

In [65]:
ft.wv.most_similar('школа')

[('школа»', 0.9956318736076355),
 ('«школа', 0.9949550628662109),
 ('школы»', 0.9881325960159302),
 ('эркола', 0.9765649437904358),
 ('школе»', 0.9650436639785767),
 ('анкола', 0.9616413116455078),
 ('кока-кола', 0.9542515873908997),
 ('бизнес-школа', 0.9448583722114563),
 ('профтехшкола', 0.9263396263122559),
 ('пенсакола', 0.9237112998962402)]

**Вывод:** в данном случае, для модели Word2Vec получились более качественные эмбеддинги. Для FastText была взята слишком большая длина n-gram (минимум 4 символа), из-за чего модель стала просто находить однокоренные слова, между которыми может совсем отсутствовать связь: город -> горох, школа -> кока-кола...

# Задание 3 (4 балла)

Используя датасет для классификации (labeled.csv) и простую нейронную сеть (последняя модель в семинаре), оцените качество полученных эмбедингов в задании 1 и 2 (4 набора эмбедингов), также проверьте 1 любую из предобученных моделей с rus-vectores (но только не tayga_upos_skipgram_300_2_2019). 
Какая модель показывает наилучший результат?

Убедитесь, что для каждой модели вы корректно воспроизводите пайплайн предобработки (в 1 задании у вас лемматизация, не забудьте ее применить к датасету для классификации; у выбранной предобученной модели может быть своя специфичная предобработка - ее нужно воспроизвести)

#### Предобработка

In [66]:
data = pd.read_csv('labeled.csv')

In [67]:
# берем предобработку из задания 1
data['norm_text'] = data.comment.apply(preprocess)

In [70]:
vocab = Counter()

for text in data['norm_text']:
    vocab.update(text)
    
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)

len(filtered_vocab)

6309

In [71]:
word2id2 = { 'PAD':0}

for word in filtered_vocab:
    word2id2[word] = len(word2id2)
id2word2 = {i:word for word, i in word2id2.items()}

Переводим слова в индексы

In [134]:
X_glob = []

for tokens in data['norm_text']:
    ids = [word2id2[token] for token in tokens if token in word2id2]
    X_glob.append(ids)

#### Classification with custom embeddings

Пайплан для моделей из Gensim (задание 2):

In [160]:
def classify_with_embeddings_gensim(word2id, emb_model, vec_size):
  X = tf.keras.preprocessing.sequence.pad_sequences(X_glob, maxlen=vec_size)
  y = data.toxic.values

  # разбиваем датасет на обучающую и тестовую выборки
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)
  # создаем матрицу с векторными представлениями
  weights = np.zeros((len(word2id), vec_size))

  for word, i in word2id.items():
      # вектор паддинга оставим нулевым
      if word == 'PAD':
          continue
      try:
          weights[i] = emb_model.wv[word]
      
      
      except KeyError:
          # для слов, которых нет в модели возьмем  рандомный вектор
          continue
          weights[i] = emb_model.wv['ллалалаллала']

  inputs = tf.keras.layers.Input(shape=(vec_size,))

  # передаем матрицу в эмбединг слой
  embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=vec_size, 
                                        trainable=False,
                                        weights=[weights])(inputs, )
  mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)

  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  optimizer = tf.keras.optimizers.Adam()
  model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  # обучаем модель
  model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=30)
  
  return model

Пайплайн для моделей на TensoFlow (задание 1)

In [156]:
def classify_with_embeddings_tf(id2word, emb_model, vec_size):
  X = tf.keras.preprocessing.sequence.pad_sequences(X_glob, maxlen=vec_size)
  y = data.toxic.values

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)
  weights = np.zeros((len(word2id), vec_size))

  # получаем веса из последнего слоя TF модели (в нашем случае берем 2-й слой)
  model_weights = emb_model.layers[2].get_weights()[0]

  # тут используем id2word, тк вытаскиваем вектора из tf-модели по id 
  for id in id2word.keys(): 
      if id == 0:
          continue
      try:
          weights[id] = model_weights[id]
      
      
      except KeyError:
          # для слов, которых нет в модели возьмем  рандомный вектор
          continue
          weights[id] = model_weights[42424242424242] 

  inputs = tf.keras.layers.Input(shape=(vec_size,))

  # передаем матрицу в эмбединг слой
  embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=vec_size, 
                                        trainable=False,
                                        weights=[weights])(inputs, )
  mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)

  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  optimizer = tf.keras.optimizers.Adam()
  model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  # обучаем модель
  model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=30)
  
  return model

#### Сравнение полученных классификаторов

| Classifier | Accuracy on final epoch |
| --- | --- |
| Fasttext Classifier | 0.72 |
| Word2Vec Classifier | 0.69 |
| CBOW Negative Sampling | 0.6653 |
| Skip-gram Negative Sampling | 0.6656 |



Не совсем понятно, почему 3 и 4 модели (CBOW и Skip-gram из задания 1) показали почти одинаковые результаты при классификации, может что-то не так сделала в *classify_with_embeddings_tf()*

#### FastText classifier

In [ ]:
fasttext_classifier = classify_with_embeddings_gensim(word2id2, ft, 100)

In [162]:
fasttext_classifier.history.history['accuracy'][29]

0.7199620008468628

#### Word2Vec Classifier

In [ ]:
word2vec_classifier = classify_with_embeddings_gensim(word2id2, w2v, 128)

In [164]:
word2vec_classifier.history.history['accuracy'][29]

0.6998758316040039

#### CBOW Negative Sampling Classifier

In [ ]:
id2word

In [ ]:
cbow_classifier = classify_with_embeddings_tf(id2word2, model, 300)

In [158]:
cbow_classifier.history.history['accuracy'][29]

0.665254533290863

#### Skip-gram Negative Sampling Classifier

In [166]:
skipgram_classifier = classify_with_embeddings_tf(id2word2, skipgram_neg_smp_model, 300)

Epoch 1/30
428/428 [==============================] - 4s 7ms/step - loss: 0.6629 - accuracy: 0.6655 - val_loss: 0.6498 - val_accuracy: 0.6560
Epoch 2/30
428/428 [==============================] - 2s 4ms/step - loss: 0.6365 - accuracy: 0.6656 - val_loss: 0.6414 - val_accuracy: 0.6560
Epoch 3/30
428/428 [==============================] - 2s 4ms/step - loss: 0.6302 - accuracy: 0.6656 - val_loss: 0.6404 - val_accuracy: 0.6560
Epoch 4/30
428/428 [==============================] - 2s 4ms/step - loss: 0.6283 - accuracy: 0.6656 - val_loss: 0.6405 - val_accuracy: 0.6560
Epoch 5/30
428/428 [==============================] - 2s 4ms/step - loss: 0.6274 - accuracy: 0.6656 - val_loss: 0.6404 - val_accuracy: 0.6560
Epoch 6/30
428/428 [==============================] - 2s 4ms/step - loss: 0.6266 - accuracy: 0.6656 - val_loss: 0.6404 - val_accuracy: 0.6560
Epoch 7/30
428/428 [==============================] - 2s 4ms/step - loss: 0.6259 - accuracy: 0.6656 - val_loss: 0.6404 - val_accuracy: 0.6560
Epoch 

In [167]:
skipgram_classifier.history.history['accuracy'][29]

0.6656197309494019

#### Ruscorpora_upos_cbow Classifier

In [168]:
rusvec_model = gensim.models.KeyedVectors.load_word2vec_format('ruscorpora_upos_cbow_300_20_2019.bin', binary=True)

Чтобы взять эмбеддинги из этой модели, нужно сделать препроцессинг с POS-tag разметкой (но на этом я решила остановиться))

In [170]:
for n in rusvec_model.most_similar(positive=[u'школа_NOUN']):
    print(n[0], n[1])

школа_PROPN 0.7144617438316345
училище_NOUN 0.6902965307235718
гимназия_NOUN 0.6282987594604492
общеобразовательный_ADJ 0.6118607521057129
интернат_NOUN 0.6072897911071777
обучение_NOUN 0.6067934036254883
педагог_NOUN 0.6042443513870239
колледж_NOUN 0.6035988330841064
профтехучилищ_NOUN 0.5997089147567749
пту_NOUN 0.5955549478530884
